In [ ]:
import xarray as xr

In [ ]:
papa_data = xr.load_dataset(
    "../stations/papa/2_processed/papa_zooplankton.zarr", engine="zarr"
)
papa_data

In [ ]:
results = xr.Dataset(
    {
        "day": papa_data["total"].sel(is_day=True).mean(["depth"]).drop("is_day"),
        "night": papa_data["total"].sel(is_day=False).mean(["depth"]).drop("is_day"),
    }
)
results

In [ ]:
import plotly.express as px

day_df = (
    results["day"].mean(["latitude", "longitude"]).to_dataframe().reset_index().dropna()
)
night_df = (
    results["night"]
    .mean(["latitude", "longitude"])
    .to_dataframe()
    .reset_index()
    .dropna()
)


fig = px.line(
    title="Day and Night Series", labels={"value": "Abundance", "time": "Time"}
)

fig.add_scatter(x=day_df["time"], y=day_df["day"], mode="lines", name="Day")

fig.add_scatter(x=night_df["time"], y=night_df["night"], mode="lines", name="Night")

fig.update_layout(
    xaxis_title="Time",
    yaxis_title="Wet weight mg/m3",
    legend_title="Day/Night",
    legend=dict(orientation="h"),
)

fig.show()

In [ ]:
data_export = (
    xr.Dataset(
        {
            "day": papa_data["total"]
            .sel(is_day=True)
            .mean(["depth"])
            .drop_vars("is_day"),
            "night": papa_data["total"]
            .sel(is_day=False)
            .mean(["depth"])
            .drop_vars("is_day"),
        }
    )
    .expand_dims({"layer": [0]})
    .transpose("time", "latitude", "longitude", "layer")
)
data_export["day"].attrs = {
    "description": "Zooplankton dry weight by cube meter of sea water at day",
    "units": "mg/m3",
}
data_export["night"].attrs = {
    "description": "Zooplankton dry weight by cube meter of sea water at night",
    "units": "mg/m3",
}
data_export["layer"].attrs = {
    "flag_values": [0, 1, 2],
    "flag_meanings": "epipelagic, upper-mesopelagic, lower-mesopelagic",
    "desc": "The layer depth according to Seapodym definition",
    "standard_name": "layer",
    "long_name": "Layer depth",
    "axis": "Z",
}
data_export

In [ ]:
data_export.to_zarr("Papa_zooplankton.zarr", mode="w")